# 1. Синтетическая генерация групповых изображений (автоматически)

Скрипт для создания реалистичных изображений с несколькими цифрами:

In [12]:

import cv2
import numpy as np
import random
import os

In [13]:
# Генерация чб изображений

def create_composite_image(digits_dir, output_dir, num_samples=1000):
    """Генерирует изображения с несколькими цифрами"""
    # Создаем папки для вывода
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    
    # Загрузка всех цифр
    digits = {}
    for d in range(10):
        digit_path = os.path.join(digits_dir, str(d))
        if not os.path.exists(digit_path):
            continue
            
        digit_files = [f for f in os.listdir(digit_path) if f.endswith('.png')]
        digits[d] = [cv2.imread(os.path.join(digit_path, f), 0) for f in digit_files]                      # 0 - загрузка в grayscale
        #digits[d] = [cv2.imread(os.path.join(digit_path, f)) for f in digit_files]                          # Загрузка цветного изображения
        #digits[d] = [cv2.cvtColor(digits[d], cv2.COLOR_BGR2RGB) for f in digit_files]     # Правильный порядок каналов
        digits[d] = [img for img in digits[d] if img is not None]  # Фильтрация None
    
    for i in range(num_samples):
        # Создаем фон (белый или случайный)
        bg = np.ones((640, 640), dtype=np.uint8) * np.random.randint(200, 256)
        
        # Выбираем 1-5 случайных цифр
        num_digits = random.randint(1, 5)
        bboxes = []  # Будет хранить (x, y, w, h, label_str)
        
        for _ in range(num_digits):
            # Выбираем случайную цифру (пропускаем отсутствующие)
            available_digits = [d for d in digits if digits[d]]
            if not available_digits:
                continue
                
            digit_class = random.choice(available_digits)
            digit_img = random.choice(digits[digit_class])
            
            # # Ресайз с сохранением пропорций
            # scale = random.uniform(0.2, 0.5)
            scale = 1
            h, w = digit_img.shape
            new_w, new_h = int(w*scale), int(h*scale)
            # digit_img = cv2.resize(digit_img, (new_w, new_h))
            
            # Пытаемся разместить цифру без пересечений
            for attempt in range(20):  # Максимум 20 попыток
                x = random.randint(0, 640 - new_w)
                y = random.randint(0, 640 - new_h)
                
                # Проверка пересечений с уже размещенными цифрами
                collision = False
                for bbox in bboxes:
                    bx, by, bw, bh, _ = bbox  # Распаковываем 5 значений
                    if (x < bx + bw and x + new_w > bx and
                        y < by + bh and y + new_h > by):
                        collision = True
                        break
                
                if not collision:
                    # Накладываем цифру (инвертируем для темных цифр на светлом фоне)
                    roi = bg[y:y+new_h, x:x+new_w]
                    bg[y:y+new_h, x:x+new_w] = cv2.bitwise_and(roi, digit_img)
                    
                    # Формируем строку для YOLO
                    x_center = (x + new_w/2) / 640
                    y_center = (y + new_h/2) / 640
                    width = new_w / 640
                    height = new_h / 640
                    label_str = f"{digit_class} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    
                    # Сохраняем bbox (x, y, w, h, label_str)
                    bboxes.append((x, y, new_w, new_h, label_str))
                    break
        
        # Сохраняем только если есть хотя бы одна цифра
        if bboxes:
            # Сохраняем изображение
            img_path = os.path.join(output_dir, 'images', f'composite_{i:04d}.jpg')
            cv2.imwrite(img_path, bg)
            
            # Сохраняем разметку
            label_path = os.path.join(output_dir, 'labels', f'composite_{i:04d}.txt')
            with open(label_path, 'w') as f:
                f.write("\n".join([bbox[4] for bbox in bboxes]))

In [14]:
# Генерация датасета в цвете

def create_composite_image(digits_dir, output_dir, num_samples=1000):
    """Генерирует цветные изображения с несколькими цифрами"""
    # Создаем папки для вывода
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    
    # Загрузка всех цифр (теперь в цвете)
    digits = {}
    for d in range(10):
        digit_path = os.path.join(digits_dir, str(d))
        if not os.path.exists(digit_path):
            continue
            
        digit_files = [f for f in os.listdir(digit_path) if f.endswith('.png')]
        digits[d] = [cv2.imread(os.path.join(digit_path, f)) for f in digit_files]  # Загрузка в BGR (цветной режим)
        digits[d] = [img for img in digits[d] if img is not None]  # Фильтрация None
    
    for i in range(num_samples):
        # Создаем цветной фон (белый или случайный)
        bg = np.ones((640, 640, 3), dtype=np.uint8) * np.random.randint(200, 256, size=3)
        
        # Выбираем 3 - 6 случайных цифр
        num_digits = random.randint(1, 5)
        bboxes = []  # Будет хранить (x, y, w, h, label_str)
        
        for _ in range(num_digits):
            # Выбираем случайную цифру (пропускаем отсутствующие)
            available_digits = [d for d in digits if digits[d]]
            if not available_digits:
                continue
                
            digit_class = random.choice(available_digits)
            digit_img = random.choice(digits[digit_class])
            
            # Ресайз с сохранением пропорций
            scale = random.uniform(0.5, 1)
            h, w, _ = digit_img.shape  # Теперь 3 канала (h, w, c)
            new_w, new_h = int(w * scale), int(h * scale)
            digit_img = cv2.resize(digit_img, (new_w, new_h))
            
            # Пытаемся разместить цифру без пересечений
            for attempt in range(20):  # Максимум 20 попыток
                x = random.randint(0, 640 - new_w)
                y = random.randint(0, 640 - new_h)
                
                # Проверка пересечений с уже размещенными цифрами
                collision = False
                for bbox in bboxes:
                    bx, by, bw, bh, _ = bbox
                    if (x < bx + bw and x + new_w > bx and
                        y < by + bh and y + new_h > by):
                        collision = True
                        break
                
                if not collision:
                    # Накладываем цифру (используем альфа-канал, если он есть)
                    if digit_img.shape[2] == 4:  # Если есть альфа-канал (PNG с прозрачностью)
                        alpha = digit_img[:, :, 3] / 255.0
                        for c in range(3):
                            bg[y:y+new_h, x:x+new_w, c] = (
                                bg[y:y+new_h, x:x+new_w, c] * (1 - alpha) + 
                                digit_img[:, :, c] * alpha
                            ).astype(np.uint8)
                    else:  # Если нет альфа-канала, просто копируем
                        bg[y:y+new_h, x:x+new_w] = digit_img
                    
                    # Формируем строку для YOLO
                    x_center = (x + new_w / 2) / 640
                    y_center = (y + new_h / 2) / 640
                    width = new_w / 640
                    height = new_h / 640
                    label_str = f"{digit_class} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    
                    # Сохраняем bbox (x, y, w, h, label_str)
                    bboxes.append((x, y, new_w, new_h, label_str))
                    break
        
        # Сохраняем только если есть хотя бы одна цифра
        if bboxes:
            # Сохраняем изображение
            img_path = os.path.join(output_dir, 'images', f'composite_{i:04d}.jpg')
            cv2.imwrite(img_path, bg)
            
            # Сохраняем разметку
            label_path = os.path.join(output_dir, 'labels', f'composite_{i:04d}.txt')
            with open(label_path, 'w') as f:
                f.write("\n".join([bbox[4] for bbox in bboxes]))

In [15]:
# Путь к новому датасету
dataset_path = '/home/lastinm/PROJECTS/DATA/syntetic digits'

In [16]:
# Использование
create_composite_image("/home/lastinm/PROJECTS/DATA/digits/images",
                       dataset_path,
                       num_samples=5000)

[ WARN:0@2552.363] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


# Разделение на train/val

In [17]:
import random
from shutil import copyfile

In [18]:

images_dir = f"{dataset_path}/images"
labels_dir = f"{dataset_path}/labels"

print(f"{images_dir}, {labels_dir}")

/home/lastinm/PROJECTS/DATA/syntetic digits/images, /home/lastinm/PROJECTS/DATA/syntetic digits/labels


In [19]:
# После генерации всех данных:
all_images = os.listdir(images_dir)
random.shuffle(all_images)

# 80% train, 20% val
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

# Создаем подпапки
os.makedirs(f"{dataset_path}/train/images", exist_ok=True)
os.makedirs(f"{dataset_path}/train/labels", exist_ok=True)
os.makedirs(f"{dataset_path}/val/images", exist_ok=True)
os.makedirs(f"{dataset_path}/val/labels", exist_ok=True)

# Копируем файлы
for img_name in train_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join(f"{dataset_path}/train/images", img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.jpg', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join(f"{dataset_path}/train/labels", label_name)
    copyfile(src, dst)

# Аналогично для val...
for img_name in val_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join(f"{dataset_path}/val/images", img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.jpg', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join(f"{dataset_path}/val/labels", label_name)
    copyfile(src, dst)